In [1]:
from random import choices, choice, shuffle
from typing import List
from datetime import date

from arr import Contract, ContractHeader, ContractLine

import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta as rd

In [2]:
MIN_DATE = date(2020, 1, 1)
MAX_DATE = date(2026, 12, 31)
START_DATES = pd.date_range(MIN_DATE, MAX_DATE)
RENEWAL_CHANCE = .8
EXPANSION_CHANCE = .1
DOWNGRADE_CHANCE = .1

CUSTOMERS = pd.read_excel('data/saas_corp.xlsx','customer')
CUSTOMERS.set_index('key', inplace=True)

PRODUCTS = pd.read_excel('data/saas_corp.xlsx','product')
PRODUCTS.set_index('key', inplace=True)
PRODUCT_WEIGHTS = np.arange(0,1,1/len(PRODUCTS))[::-1]
PRODUCT_COUNT = len(PRODUCTS['product_name'].unique())

CONTRACTS = pd.read_excel('data/saas_corp.xlsx', 'contract')

In [3]:
def random_contract_range(
    contract_lengths: List[int] = [3, 6, 12, 24, 36],
    contract_weights: List[float] = [0.1, 0.05, 0.6, 0.2, 0.05],
) -> int:
    """Get a random contract length interval.

    Wrapper for:
    https://docs.python.org/3/library/random.html#random.choices

    Args:
        contract_lengths (List[int], optional): Possible contract lengths, in months.
            Defaults to [3, 6, 12, 24, 36].
        contract_weights (List[float], optional): Weights of the `contract_lengths`, arg.
            Defaults to [0.1, 0.05, 0.6, 0.2, 0.05].

    Returns:
        int: Length of contract in months.
    """
    return choices(contract_lengths, contract_weights)[0]

In [4]:
def initial_sale(customer: str) -> Contract:
    """Get customer start, end dates. Get customer items for first sale"""

    start_date = choice(START_DATES).date()
    contract_range = random_contract_range()
    end_date = start_date + rd(months=contract_range, days=-1)

    num_of_items = choices(range(1, PRODUCT_COUNT + 1), PRODUCT_WEIGHTS)[0]
    products = PRODUCTS.loc[np.random.choice(PRODUCTS.index, num_of_items, False)]

    contract = Contract(
        id=1,
        header=ContractHeader(products["amount"].sum(), start_date, end_date),
        lines=[
            ContractLine(
                row[1]["amount"],
                start_date,
                end_date,
                row[1]["product_name"],
                row[1]["renewable"],
            )
            for row in products.iterrows()
        ],
        customer=customer,
    )

    return contract

In [5]:
for customer in CUSTOMERS['customer'].unique():
    if customer in CONTRACTS["customer"].unique():
        print(f"{customer} already in contract list")
    else:
        print(initial_sale(customer))

Abatz already in contract list
                         Topicshots                         
             Contract #1 - $38,000 - 2026-04-26             
                  2026-04-26 - 2028-04-25                   
************************************************************
product	start date	end date	amount	renewable
************************************************************
LMS	2026-04-26	2028-04-25	$13,000	True
ERP	2026-04-26	2028-04-25	$25,000	True
************************************************************

                           Twinte                           
             Contract #1 - $23,000 - 2025-02-22             
                  2025-02-22 - 2026-02-21                   
************************************************************
product	start date	end date	amount	renewable
************************************************************
VPN	2025-02-22	2026-02-21	$10,000	True
HRIS	2025-02-22	2026-02-21	$13,000	True
**************************************************